In [8]:
# nuclio: ignore
# do not remove the comment above (it is a directive to nuclio, ignore that cell during build)
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter and restart the kernel 
import nuclio 

We use `%nuclio` magic commands to set package dependencies and configuration:

In [9]:
%nuclio cmd -c pip install pandas
%nuclio config spec.build.baseImage = "mlrun/mlrun"

%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'


The ```DataItem```s and the ```context``` within which they are logged are described in the following ```mlrun``` modules (they are included here only for type clarity).

In [10]:
from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem

______________________________________________

In [11]:
from mlrun import run_local, code_to_function, mlconf, NewTask
from mlrun.platforms import mount_pvc
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'

<b> define the artifact location</b>

In [12]:
from os import path
out = mlconf.artifact_path or path.abspath('./data')
# {{run.uid}} will be substituted with the run id, so output will be written to different directoried per run
artifact_path = path.join(out, '{{run.uid}}')

#### _running and linking multiple tasks_
In this example we run two functions, ```training``` and ```validation``` and we pass the result from one to the other.
We will see in the ```job``` example that linking works even when the tasks are run in a workflow on different processes or containers.

```run_local()``` will run our task on a local function:

Run the training function. Functions can have multiple handlers/methods, here we call the ```training``` handler:

In [13]:
train_run = run_local(NewTask(handler='training', params={'p1': 5}, artifact_path=out),command='functions.py')

[mlrun] 2020-04-12 15:09:06,154 starting run functions-training uid=a1a512d762f7427a872c85c07656f387  -> http://mlrun-api:8080
%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'
[mlrun] 2020-04-12 15:09:06,179 starting run mlrun-5aca8d-training uid=3d4bfee1a22e4661bff075c5c818c9ee  -> http://mlrun-api:8080
Run: mlrun-5aca8d-training (uid=3d4bfee1a22e4661bff075c5c818c9ee)
Params: p1=5, p2=2
[mlrun] 2020-04-12 15:09:06,208 started training
[mlrun] 2020-04-12 15:09:06,218 log artifact model at /home/jovyan/mlrun/model.txt, size: 10, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...18c9ee,0,Apr 12 15:09:06,completed,mlrun-5aca8d-training,kind=handlerowner=jovyanhost=jupyter-notebook-5c687f7ccc-dptgvcategory=tests,,p1=5,accuracy=10loss=15,model


to track results use .show() or .logs() or in CLI: 
!mlrun get run 3d4bfee1a22e4661bff075c5c818c9ee --project default , !mlrun logs 3d4bfee1a22e4661bff075c5c818c9ee --project default
[mlrun] 2020-04-12 15:09:06,260 run executed, status=completed
[mlrun] 2020-04-12 15:09:06,269 starting run mlrun-0304b2-validation uid=8463ffe12a0741f4943429f324c70be6  -> http://mlrun-api:8080
Run: mlrun-0304b2-validation (uid=8463ffe12a0741f4943429f324c70be6)
file - /home/jovyan/mlrun/model.txt:
b'abc is 123'

[mlrun] 2020-04-12 15:09:06,293 started validation
[mlrun] 2020-04-12 15:09:06,303 log artifact validation at /home/jovyan/mlrun/validation.html, size: 18, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...c70be6,0,Apr 12 15:09:06,completed,mlrun-0304b2-validation,kind=handlerowner=jovyanhost=jupyter-notebook-5c687f7ccc-dptgv,model,,,validation


to track results use .show() or .logs() or in CLI: 
!mlrun get run 8463ffe12a0741f4943429f324c70be6 --project default , !mlrun logs 8463ffe12a0741f4943429f324c70be6 --project default
[mlrun] 2020-04-12 15:09:06,353 run executed, status=completed
Run: functions-training (uid=a1a512d762f7427a872c85c07656f387)
Params: p1=5, p2=2
[mlrun] 2020-04-12 15:09:06,366 started training
[mlrun] 2020-04-12 15:09:06,378 log artifact model at /home/jovyan/mlrun/model.txt, size: 10, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...56f387,0,Apr 12 15:09:06,completed,functions-training,kind=owner=jovyanhost=jupyter-notebook-5c687f7ccc-dptgvcategory=tests,,p1=5,accuracy=10loss=15,model


to track results use .show() or .logs() or in CLI: 
!mlrun get run a1a512d762f7427a872c85c07656f387 --project default , !mlrun logs a1a512d762f7427a872c85c07656f387 --project default
[mlrun] 2020-04-12 15:09:06,419 run executed, status=completed


After the function runs it generates the result widget, you can click the `model` artifact to see its content.

In [14]:
train_run.outputs

{'accuracy': 10, 'loss': 15, 'model': '/home/jovyan/mlrun/model.txt'}

The output from the first training function is passed to the validation function, let's run it:

In [ ]:
model_path = train_run.outputs['model']

validation_run = run_local(NewTask(handler='validation', inputs={'model': model_path}, artifact_path=out),command='functions.py')